In [1]:
!nvidia-smi

Sun Jan 23 14:10:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

     |████████████████████████████████| 3.4 MB 31.5 MB/s 
     |████████████████████████████████| 3.3 MB 73.0 MB/s 
     |████████████████████████████████| 895 kB 68.2 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 69.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 248 kB 36.0 MB/s 
     |████████████████████████████████| 9.3 MB 54.8 MB/s 
     |████████████████████████████████| 311 kB 74.2 MB/s 
     |████████████████████████████████| 1.7 MB 58.6 MB/s 
     |████████████████████████████████| 1.2 MB 69.3 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 180 kB 53.5 MB/s 
     |████████████████████████████████| 97 kB 9.5 MB/s 
     |████████████████████████████████| 142 kB 75.5 MB/s 
     |████████████████████████████████| 63 kB

In [4]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import LabelEncoder

In [5]:
model = ClassificationModel('bert', 'recobo/chemical-bert-uncased', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 82,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'early_stopping_delta': 0.01,
'early_stopping_metric':"auroc",
'early_stopping_metric_minimize' : False,
'early_stopping_patience' : 10,
'max_seq_length' : 128,
'num_train_epochs' : 10,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=2,)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at recobo/chemical-bert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at recobo/chemical-

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [7]:
data = pd.read_table('/content/B3DB_classification.tsv')
data = data[pd.notnull(data['IUPAC_name'])]
X = data['IUPAC_name']
target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [8]:
y_classification.shape

(5262,)

In [9]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [12]:
Xtrain = {'text':xtrain, 'labels':ytrain}
Xvalid = {'text':xvalid, 'labels':yvalid}
Xtest = {'text':xtest, 'labels':ytest}
train_df = pd.DataFrame(Xtrain)
valid_df = pd.DataFrame(Xvalid)
test_df = pd.DataFrame(Xtest)

In [13]:
xtrain

4015    (1r,2r,3r)-2-methylsulfonyl-3-phenylcyclopropa...
3799    (2s,3s,4s,5s,6s)-2-(hydroxymethyl)-6-[2-(hydro...
1629    ethyl 2-(dimethylamino)-1-phenylcyclohex-3-ene...
3529    (9s,13s)-1,13-dimethyl-10-(2-phenylethyl)-10-a...
752     7-(2,4-dichlorophenyl)-2-methyl-4-[3-(trifluor...
                              ...                        
4190    3-(acetyloxymethyl)-8-oxo-7-[(2-thiophen-2-yla...
1793    (4s,4as,5as,6s,12ar)-n-[[2-[[[(4s,4as,5as,6s,1...
6018    [2-[(8s,9r,10s,11r,13s,14s,17r)-9-fluoro-11,17...
4766    [(2s,5r)-3,3-dimethyl-4,4,7-trioxo-4lambda6-th...
5864    n-[4-(2-methyl-4,5-dihydro-3h-imidazo[4,5-d][1...
Name: IUPAC_name, Length: 3535, dtype: object

In [14]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [15]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'bbb_model_iupac'})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3535 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/674 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.637937121849701, 'tp': 185, 'tn': 375, 'fp': 39, 'fn': 75, 'auroc': 0.8986575622445189, 'auprc': 0.8587989815748678, 'eval_loss': 1.085889263889369}
INFO:simpletransformers.classification.classification_model: No improvement in auroc
INFO:simpletransformers.classification.classification_model: Current step: 9
INFO:simpletransformers.classification.classification_model: Early stopping patience: 10
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/674 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6319369938918088, 'tp': 188, 'tn': 370, 'fp': 44, 'fn': 72, 'auroc': 0.8994565217391304, 'auprc': 0.8591388008209074, 'eval_loss': 1.0665602764662574}
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [16]:

model = ClassificationModel(
    "bert", "/content/outputs/best_model",args={'wandb_project': 'bbb_model_iupac'}
)

In [17]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1053 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/132 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,█▆▅▆▄▃▂▂▂▁▂▁▃▁▁▁▁▁▁▂▁▁
auprc,▁▃▆▇▇▇▇█▆▇▇▇▆▇▅▅▅▆▆▆▆▆▆
auroc,▁▄▆▇▇███▇▇▇▇▇▇▆▅▅▆▆▆▆▆▆
eval_loss,▂▂▁▁▁▂▁▂▂▃▃▃▅▅▆▇▇█▇▇███
fn,▆█▃▃▃▆▅▃▃▁▂▂▄▂▃▃▃▄▃▂▃▃▃
fp,▇▄▅▅▆▁▃▆▆█▇█▄█▇▆▆▆███▆▇
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr,▆██▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
mcc,▂▁█▇▆▇▇▇▆▇▇▆▆▆▆▆▆▅▅▆▅▆▅
tn,▂▅▄▄▃█▆▃▃▁▂▁▅▁▂▃▃▃▁▁▁▃▂
tp,▃▁▆▅▆▃▅▇▆█▇▇▅▇▆▆▆▅▆▇▇▆▆


INFO:simpletransformers.classification.classification_model:{'mcc': 0.623095954693168, 'tp': 248, 'tn': 620, 'fp': 35, 'fn': 150, 'auroc': 0.9022766504277111, 'auprc': 0.868543136422235, 'acc': 0.8243114909781576, 'eval_loss': 0.48887138672624575}


{'mcc': 0.623095954693168, 'tp': 248, 'tn': 620, 'fp': 35, 'fn': 150, 'auroc': 0.9022766504277111, 'auprc': 0.868543136422235, 'acc': 0.8243114909781576, 'eval_loss': 0.48887138672624575}


In [18]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/674 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/85 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6577751156822162, 'tp': 161, 'tn': 402, 'fp': 12, 'fn': 99, 'auroc': 0.9143859160163508, 'auprc': 0.8778023719467597, 'acc': 0.8353115727002968, 'eval_loss': 0.4626305425868315}


{'mcc': 0.6577751156822162, 'tp': 161, 'tn': 402, 'fp': 12, 'fn': 99, 'auroc': 0.9143859160163508, 'auprc': 0.8778023719467597, 'acc': 0.8353115727002968, 'eval_loss': 0.4626305425868315}


In [19]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,972 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories c

In [20]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [21]:
from transformers import AutoModel
from transformers import AutoTokenizer

model = AutoModel.from_pretrained("/content/outputs/best_model")
tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")

Some weights of the model checkpoint at /content/outputs/best_model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
model.push_to_hub("BBB_prediction_finetuned_classification_iupac")
tokenizer.push_to_hub("BBB_prediction_finetuned_classificationa_iupac")

Cloning https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.36k/419M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac
   bad11b6..a68e916  main -> main

   bad11b6..a68e916  main -> main

Cloning https://huggingface.co/Parsa/BBB_prediction_finetuned_classificationa_iupac into local empty directory.
To https://huggingface.co/Parsa/BBB_prediction_finetuned_classificationa_iupac
   f1ec4ab..2e5dd58  main -> main

   f1ec4ab..2e5dd58  main -> main



'https://huggingface.co/Parsa/BBB_prediction_finetuned_classificationa_iupac/commit/2e5dd58ddc6540a47778e9bc550a53a7508a7fd9'